In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nechbamohammed/research-papers-dataset")

print("Path to dataset files:", path)
import os
csv_file=os.listdir(path)
print(csv_file[0])

csv_file_path = os.path.join(path, csv_file[0])
print(csv_file_path)

/opt/anaconda3/envs/py_proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 590M/590M [02:22<00:00, 4.34MB/s] 

Extracting files...


Path to dataset files: /Users/anuragprasad/.cache/kagglehub/datasets/nechbamohammed/research-papers-dataset/versions/1
dblp-v10.csv
/Users/anuragprasad/.cache/kagglehub/datasets/nechbamohammed/research-papers-dataset/versions/1/dblp-v10.csv
